## Model and Config Setup

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_vision_completion, get_image_generation_completion, get_completion, save_artifact, clean_llm_output, recommended_models_table
from IPython.display import Image, display, Code

image_client, image_model_name, image_api_provider = setup_llm_client(model_name="gemini-2.5-flash-image-preview")
print(f"✅ Using {image_model_name} for image generation")

# Ensure you select a vision-capable model
vision_client, vision_model_name, vision_api_provider = setup_llm_client(model_name="gemini-2.5-pro")
print(f"✅ Using {vision_model_name} for vision tasks")


if not image_model_name or not vision_model_name:
    print("Could not set up a valid LLM client. Please check your .env file and utils.py configuration.")

## Get the Desired UI Screenshot for View

In [ ]:
#TODO: INSERT PROMPT FOR DESIRED VIEW GENERATION
screen_generation_prompt = """
"""

print(f"Using {image_model_name} for image generation...")
file_path, image_url = get_image_generation_completion(screen_generation_prompt, image_client, image_model_name, image_api_provider)

# Display the generated image if successful
if file_path and image_url:
    print(f"✅ Generated image saved to: {file_path}")
    display(Image(url=image_url))
elif image_url:
    print(f"❌ Image generation failed: {image_url}")
else:
    print("❌ Image generation returned no result")

## Generate Monolithic UI Code

First, generate an explanation of the image to pass to the model

In [ ]:
image_path = file_path

explain_image_prompt = f"""
You are an expert frontend developer and UI/UX designer. Analyze the provided image and produce a clear, actionable explanation for a developer who will implement the UI.

Please respond in markdown and include the following sections:

1) Summary: One-sentence summary of what this screen is (purpose).
2) Visual Breakdown: Enumerate the visible UI elements (e.g., header, logo, form fields, buttons, icons, images) and approximate positions (top/left/center, grouping).
3) Style Details: Describe colors, font styles (weight/size/line-height if evident), spacing, borders/radii, shadows, and alignment cues.
4) Interaction & Behavior: Identify interactive elements and the likely behaviors (hover/focus states, validation, keyboard affordances).
5) Accessibility Notes: Potential accessibility issues and recommended fixes (labels, contrast, focus order).
6) Implementation Plan (React + Tailwind): Suggest component names and a short mapping of each component to Tailwind utilities or structure (e.g., <LoginForm> -> container: flex, gap-4, ...). Keep this as a concise checklist.

Keep the output developer-focused and actionable.
"""

print("--- Requesting image explanation from vision model ---")
if vision_model_name:
    explanation = get_vision_completion(explain_image_prompt, image_path, vision_client, vision_model_name, vision_api_provider)
    cleaned_explanation = clean_llm_output(explanation, language="markdown")
    display(Code(cleaned_explanation, language="markdown"))
    # Save the explanation for later reference
    save_artifact(cleaned_explanation, "artifacts/image_explanation.md")
else:
    print("Skipping image explanation because no vision model is configured.")

Now, generate the monolithic front-end code for the generated view

In [ ]:
from utils import load_artifact
image_explanation = load_artifact("image_explanation.md")
generate_ui_prompt = f"""
You are an expert frontend developer specializing in React and Tailwind CSS. Your task is to analyze the provided design image and generate a single, self-contained React component that precisely replicates the visual design.
You also have access to an image explanation generated by a previous model: 
{image_explanation}

**Requirements:**
1. Analyze the image carefully, paying attention to:
   - Layout structure, spacing, and alignment
   - Colors, typography, font sizes, and font weights
   - Border radii, shadows, and visual effects
   - All UI elements (buttons, inputs, text, icons, images, etc.)
   - Exact positioning and grouping of elements

2. Generate a single React functional component using:
   - React functional component syntax with TypeScript (optional types)
   - Tailwind CSS utility classes for ALL styling (no inline styles or separate CSS)
   - Semantic HTML elements where appropriate
   - Proper accessibility attributes (aria-labels, roles, etc.)

3. Output format:
   - Export a single default component (e.g., `export default function HomePage()`)
   - Include all necessary imports at the top
   - Use Tailwind CSS classes exclusively for styling
   - Match colors, spacing, typography, and layout as accurately as possible
   - Include placeholder text and content that matches the design
   - Ensure responsive design considerations where visible in the image

4. Code quality:
   - Clean, readable code with appropriate comments if needed
   - Proper JSX structure and formatting
   - Use semantic HTML elements
   - Include interactive states (hover, focus) where visually indicated

**Output ONLY the React component code. Do not include explanations, markdown formatting, or additional text outside of the code.**
"""

print("--- Generating Monolithic UI Component ---")
if vision_model_name:
    generated_monolithic_code = get_vision_completion(generate_ui_prompt, image_path, vision_client, vision_model_name, vision_api_provider)
    cleaned_code = clean_llm_output(generated_monolithic_code, language='jsx')
    display(Code(cleaned_code, language='jsx'))
else:
    print("Skipping UI generation because no valid model is configured.")
    cleaned_code = ""

## Refactor in Reusable Components

In [ ]:
refactor_ui_prompt = f"""
You are a senior frontend developer specializing in React and clean code architecture. Your task is to refactor the provided monolithic React component into a well-structured, component-based architecture using best practices.

**Context - Monolithic Component Code:**
{cleaned_code}

**Requirements:**
1. Analyze the monolithic component and identify logical groupings of UI elements that can be extracted into reusable sub-components.

2. Create smaller, focused components following the Single Responsibility Principle:
   - Extract repeated UI patterns (e.g., buttons, cards, icons) into reusable components
   - Separate layout sections (e.g., header, footer, main content) into dedicated components
   - Create styled button components, navigation components, card components, etc.
   - Extract icon components if they are defined inline or can be reused

3. Component structure:
   - Each sub-component should have a clear, descriptive name (e.g., `<Header>`, `<FeatureCard>`, `<StyledButton>`, `<NavigationMenu>`)
   - Use React functional component syntax
   - Maintain all Tailwind CSS styling from the original code
   - Keep components in the same file (no separate imports needed)
   - Ensure proper prop types/parameters if components need customization

4. Main component refactoring:
   - The main component should be simplified and use the extracted sub-components
   - Maintain the exact same visual output and styling as the original
   - Use clear, semantic component names
   - Keep the same export structure (export default)

5. Code quality standards:
   - Clean, readable code with clear component boundaries
   - Proper JSX formatting and indentation
   - Comments where necessary to explain component purpose
   - Follow React best practices for component composition

**Output format:**
Output ONLY the complete refactored React code with all components defined. Include all necessary component definitions before the main component, and then the main component that uses them. Do not include explanations, markdown formatting, or additional text outside of the code.

The refactored code should demonstrate professional React component architecture while maintaining 100% visual fidelity to the original monolithic component.
"""

print("--- Refactoring UI into Components ---")
if cleaned_code:
    refactored_code = get_completion(refactor_ui_prompt, vision_client, vision_model_name, vision_api_provider)
    cleaned_refactored_code = clean_llm_output(refactored_code, language='jsx')
    display(Code(cleaned_refactored_code, language='jsx'))
else:
    print("Skipping refactoring because monolithic code was not generated.")
    cleaned_refactored_code = ""